# MNIST Training using MXNet

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

Horovod is a distributed deep learning training framework for TensorFlow, Keras, PyTorch, and MXNet. This notebook example shows how to use Horovod with MXNet in SageMaker using MNIST dataset.

For more information about the MXNet in SageMaker, please visit following github repositories:
1. [sagemaker-mxnet-training-toolkit](https://github.com/aws/sagemaker-mxnet-training-toolkit/)
2. [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) 
3. [sagemaker-training-toolkit](https://github.com/aws/sagemaker-training-toolkit)
4. [deep-learning-containers](https://github.com/aws/deep-learning-containers)

---

## Setup

_This notebook was created and tested on an ml.p2.xlarge notebook instance._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the [Amazon SageMaker Roles](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with the appropriate full IAM role arn string(s).


In [14]:
import sagemaker
from sagemaker.mxnet import MXNet

sess= sagemaker.Session()

bucket = sess.default_bucket()
prefix = 'sagemaker/DEMO-mxnet-mnist-horovod'

role = sagemaker.get_execution_role()

output_path='s3://' + sess.default_bucket() + '/' + prefix

## Data
### Getting the data

In this example, we will ues MNIST dataset. MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits).

### Uploading the data to S3

We are going to use the sagemaker.Session.upload_data function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.

In [5]:
import logging
import boto3
from botocore.exceptions import ClientError
import os
import json
# Download training and testing data from a public S3 bucket

def download_from_s3(data_dir='/tmp/data', train=True):
    """Download MNIST dataset and convert it to numpy array
    
    Args:
        data_dir (str): directory to save the data
        train (bool): download training set
    
    Returns:
        None
    """
    
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    if train:
        images_file = "train-images-idx3-ubyte.gz"
        labels_file = "train-labels-idx1-ubyte.gz"
    else:
        images_file = "t10k-images-idx3-ubyte.gz"
        labels_file = "t10k-labels-idx1-ubyte.gz"
    
    with open('code/config.json', 'r') as f:
        config = json.load(f)

    # download objects
    s3 = boto3.client('s3')
    bucket = config['public_bucket']
    for obj in [images_file, labels_file]:
        key = os.path.join("datasets/image/MNIST", obj)
        dest = os.path.join(data_dir, obj)
        if not os.path.exists(dest):
            s3.download_file(bucket, key, dest)
    return


download_from_s3('/tmp/data', True)
download_from_s3('/tmp/data', False)


# upload to the default bucket

prefix = 'mnist'
bucket = sess.default_bucket()
loc = sess.upload_data(path='/tmp/data', bucket=bucket, key_prefix=prefix)

channels = {
    "training": loc,
    "testing": loc
}

## Train

Define hyperparameters of training job. Note, that `entry_point` param defines training script which will be executed on Horovod distributed cluster. Additionally, you can also define any parameters of your training script.


### Training Scipt

The mnist.py script provides the code we need for training a SageMaker model. The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

- `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to. These artifacts are uploaded to S3 for model hosting.
- `SM_NUM_GPUS`: The number of gpus available in the current container.
- `SM_CURRENT_HOST`: The name of the current container on the container network.
- `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's fit() method, the following will be set, following the format SM_CHANNEL_[channel_name]:

- `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit SageMaker Containers.

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to model_dir so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

This script uses Horovod framework for distributed training. 

You can run the following command to view the script run by this notebook:

In [ ]:
!pygmentize horovod_mnist.py

## Run training in SageMaker

The `MXNet` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 `ml.p2.8xlarge` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)).

### SageMaker MXNet Estimator

Estimator API in Sagemaker Python SDK supports distributed training functionality via the distributions parameter.
To leverage Horovod, we specify `mpi` dictionary in the distributions parameter. The dictionary can contain following keys
- `enabled`: True/False
- `custom_mpi_options`: string
- `processes_per_host`: integer

Note: `train_instance_type` and `processes_per_host` are interlinked. Make sure that `processes_per_host` doesn't exceed the number of available GPUs in the instance.


For further details on various AWS EC2 instances & available GPUs refer:
- P3 (https://aws.amazon.com/ec2/instance-types/p3/)
- G4 (https://aws.amazon.com/ec2/instance-types/g4/)

In [17]:
mpi_options = '-verbose -x orte_base_help_aggregate=0'
distributions = {
    'mpi':{
        'enabled': True,
        'custom_mpi_options': mpi_options,
        'processes_per_host': 4
    }
}
hyperparameters = {
    'batch-size': 64,
    'dtype': 'float32',
    'epochs': 5,
    'lr': 0.02,
}

In [20]:
local_mode = True

instance_type = 'local' if local_mode else 'ml.p3.8xlarge'


estimator = MXNet(
    entry_point='train.py',
    source_dir='.',
    role=role,
    instance_type='local',
    instance_count=1,
    framework_version='1.6.0',
    output_path=output_path,
    py_version='py3',
    distributions=distributions,
    hyperparameters=hyperparameters,
    #sagemaker_session=sess
    )

distributions has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


After we've constructed our `MXNet` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.

In [21]:
estimator.fit(job_name='test-mx-horovod', inputs=channels)

Using the short-lived AWS credentials found in session. They might expire while running.


Creating tmpups3rt7q_algo-1-biljy_1 ... 
Attaching to tmpups3rt7q_algo-1-biljy_12mdone
algo-1-biljy_1  | 2020-11-27 22:27:16,559 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
algo-1-biljy_1  | 2020-11-27 22:27:16,561 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-biljy_1  | 2020-11-27 22:27:16,572 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1-biljy"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{"batch-size":64,"dtype":"float32","epochs":5,"lr":0.02}', 'SM_USER_ENTRY_POINT': 'train.py', 'SM_FRAMEWORK_PARAMS': '{"sagemaker_mpi_custom_mpi_options":"-verbose -x orte_base_help_aggregate=0","sagemaker_mpi_enabled":true,"sagemaker_mpi_num_of_processes_per_host":4}', 'SM_RESOURCE_CONFIG': '{"current_host":"algo-1-biljy","hosts":["algo-1-biljy"]}', 'SM_INPUT_DATA_CONFIG': '{"testing":{"TrainingInputMode":"File"},"training":{"TrainingInputMode":"File"}}', 

algo-1-biljy_1  | [1,1]<stderr>:INFO:root:So should it be
algo-1-biljy_1  | [1,0]<stderr>:INFO:root:So should it be
algo-1-biljy_1  | [1,2]<stderr>:INFO:root:So should it be
algo-1-biljy_1  | [1,3]<stderr>:INFO:root:So should it be
algo-1-biljy_1  | [1,2]<stdout>:Start training ...
algo-1-biljy_1  | [1,1]<stdout>:Start training ...
algo-1-biljy_1  | [1,0]<stdout>:Start training ...
algo-1-biljy_1  | [1,3]<stdout>:Start training ...
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 50000, errno = 1
algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 50000, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 50000, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 131072, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 131072, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 131072, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[

algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,

algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,

...

algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,

algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,

algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 440812, errno = 1
algo-1-biljy_1  | [1,

algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,2]<stderr>:[algo-1-biljy:00028] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,3]<stderr>:[algo-1-biljy:00029] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,0]<stderr>:[algo-1-biljy:00026] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,1]<stderr>:[algo-1-biljy:00027] Read -1, expected 440292, errno = 1
algo-1-biljy_1  | [1,

KeyboardInterrupt: 

## Host
### Create an inference endpoint

After training, we use the MXNet estimator object to build and deploy an MXNetPredictor. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

This allows us to perform inference on json encoded multi-dimensional arrays.

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances. Here we will deploy the model to a single `ml.m4.xlarge` instance.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

## Evaluate 
We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a 'data' variable in this notebook, which we can then pass to the mxnet predictor. 

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

In [ ]:
import numpy as np

image = np.array([data], dtype=np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

### Cleanup
After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [ ]:
estimator.delete_endpoint()